<a href="https://colab.research.google.com/github/njucs/med/blob/master/PM/PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 授权 Colab 访问 Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/'Colab Notebooks'/Ophthalmology/PathologicMyopia/
!pwd

/content/drive/MyDrive/Colab Notebooks/Ophthalmology/PathologicMyopia
/content/drive/MyDrive/Colab Notebooks/Ophthalmology/PathologicMyopia


In [ ]:
!ls dataset/PALM-Training400 -l

In [ ]:
!ls dataset/PALM-Training400/PALM-Training400 -l

In [ ]:
!ls dataset/PALM-Validation400 -l

### model

In [4]:
# import 导入模块，每次使用模块中的函数都要是定是哪个模块
# from … import * 导入模块，每次使用模块中的函数直接用就可以了，因为已经知道该函数是哪个模块中的了。
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision as tv
from torchvision import models,transforms,datasets

# 查看Python解释器
import sys
print(sys.executable)

# 测试GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())
use_gpu = torch.cuda.is_available()

# 把Tensor转成Image，方便可视化
from torchvision.transforms import ToPILImage
show = ToPILImage()

/usr/bin/python3
Using gpu: True 


In [ ]:
import os
import re
import random
import torch
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# 读取数据
for dirname, _, filenames in os.walk('dataset/PALM-Training400/PALM-Training400/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))